In [2]:
!pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 4.6 MB/s 


In [3]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np


In [4]:
!unzip /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata.zip -d /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata

Archive:  /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata.zip
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/1.mid  
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/10.mid  
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/11.mid  
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/12.mid  
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/13.mid  
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/14.mid  
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/15.mid  
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/16.mid  
  inflating: /content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/17.mid  
  inflating: /content/drive/MyDrive/MAC

In [9]:
import os

notes = []
for song in os.listdir("/content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata"):
  mid = MidiFile("/content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/chillhopdata/" + song)
  for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type=="note_on":
      data = msg.bytes()
      notes.append(data[1])

In [12]:
scaler = MinMaxScaler()
notes = list(scaler.fit_transform(np.array(notes).reshape(-1,1)))

In [13]:
notes = [list(note) for note in notes]

In [17]:
X = []
y= []

n_prev = 30

for i in range(len(notes) - n_prev):
  X.append(notes[i:i+n_prev])
  y.append(notes[i+n_prev])

X_test = X[-300:]
X = X[:-300]
y = y[:-300]

In [21]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))

model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))

model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))

model.add(Dense(1))

model.add(Activation("linear"))

model.summary()

optimizer = Adam(learning_rate=0.001)

model.compile(loss="mse", optimizer=optimizer)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 30, 256)           0         
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           197120    
                                                                 
 dropout_1 (Dropout)         (None, 30, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 6

In [25]:
model.fit(np.array(X),np.array(y), batch_size=16, epochs=50)

Epoch 1/50
195/195 [==============================] - 2s 10ms/step - loss: 0.0151
Epoch 2/50
195/195 [==============================] - 2s 9ms/step - loss: 0.0149
Epoch 3/50
195/195 [==============================] - 2s 9ms/step - loss: 0.0149
Epoch 4/50
195/195 [==============================] - 2s 10ms/step - loss: 0.0148
Epoch 5/50
195/195 [==============================] - 2s 10ms/step - loss: 0.0145
Epoch 6/50
195/195 [==============================] - 2s 9ms/step - loss: 0.0145
Epoch 7/50
195/195 [==============================] - 2s 9ms/step - loss: 0.0146
Epoch 8/50
195/195 [==============================] - 2s 9ms/step - loss: 0.0144
Epoch 9/50
195/195 [==============================] - 2s 9ms/step - loss: 0.0143
Epoch 10/50
195/195 [==============================] - 2s 9ms/step - loss: 0.0142
Epoch 11/50
195/195 [==============================] - 2s 10ms/step - loss: 0.0144
Epoch 12/50
195/195 [==============================] - 2s 9ms/step - loss: 0.0141
Epoch 13/50
195/195 [

In [26]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('/content/drive/MyDrive/MACHINE_LEARNING/datasets/chillhopmidi/LSTM_music.mid')